In [4]:
import tkinter as tk
from tkinter import scrolledtext
import random
import json
import numpy as np
from tensorflow.keras.models import load_model
import nltk
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
from nltk.corpus import wordnet
import pyttsx3

# Initialize pyttsx3 engine
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

# Load the trained model and data
model = load_model('chatbot_model.h5')
lemmatizer = WordNetLemmatizer()
spell = SpellChecker()

with open("intents.json") as file:
    intents = json.load(file)

# Prepare the words and classes
words = sorted(set([lemmatizer.lemmatize(w.lower()) for intent in intents['intents'] for pattern in intent['patterns'] for w in nltk.word_tokenize(pattern) if w not in ['?', '!', '.', ',']]))
classes = sorted(set(intent['tag'] for intent in intents['intents']))

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

def expand_with_synonyms(words):
    expanded_words = set(words)
    for word in words:
        synonyms = get_synonyms(word)
        expanded_words.update(synonyms)
    return list(expanded_words)

def clean_up_sentence(sentence):
    # Tokenize and lemmatize
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    
    # Expand with synonyms
    sentence_words = expand_with_synonyms(sentence_words)
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = get_response(ints, intents)
    return res

def text_to_speech(text):
    engine.say(text)
    engine.runAndWait()

def send(event=None):
    msg = entry_box.get("1.0",'end-1c').strip()
    entry_box.delete("0.0", tk.END)

    if msg != '':
        chat_log.config(state=tk.NORMAL)
        chat_log.insert(tk.END, "You: " + msg + '\n\n')
        chat_log.config(foreground="#442265", font=("Verdana", 12 ))
        
        res = chatbot_response(msg)
        chat_log.insert(tk.END, "Bot: " + res + '\n\n')
        
        # Speak the response
        text_to_speech(res)
            
        chat_log.config(state=tk.DISABLED)
        chat_log.yview(tk.END)

# Create the main window
root = tk.Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=False, height=False)

# Create chat window with integrated scrollbar
chat_log = scrolledtext.ScrolledText(root, bd=0, bg="white", height="8", width="50", font="Arial")
chat_log.config(state=tk.DISABLED)

# Create the button to send message
send_button = tk.Button(root, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,
                        bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff',
                        command=send)

# Create the box to enter message
entry_box = tk.Text(root, bd=0, bg="white", width="29", height="5", font="Arial")

# Bind the Enter key to the send function
entry_box.bind("<Return>", send)

# Place all components on the screen
chat_log.place(x=6, y=6, height=386, width=388)
entry_box.place(x=6, y=401, height=90, width=265)
send_button.place(x=275, y=401, height=90)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
